# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124120e+02     1.516338e+00
 * time: 0.3767399787902832
     1     1.044772e+01     9.245340e-01
 * time: 2.51369309425354
     2    -1.133698e+01     1.037078e+00
 * time: 3.2635891437530518
     3    -3.397678e+01     8.174619e-01
 * time: 4.32865309715271
     4    -4.728854e+01     5.925121e-01
 * time: 5.4376749992370605
     5    -5.684224e+01     2.058469e-01
 * time: 6.518611192703247
     6    -5.975199e+01     1.603956e-01
 * time: 7.227179050445557
     7    -6.090882e+01     6.490261e-02
 * time: 7.939206123352051
     8    -6.138773e+01     3.822214e-02
 * time: 8.713836193084717
     9    -6.165892e+01     3.607093e-02
 * time: 9.416200160980225
    10    -6.182796e+01     3.696621e-02
 * time: 10.131909132003784
    11    -6.198000e+01     2.451611e-02
 * time: 10.838661193847656
    12    -6.207390e+01     1.719690e-02
 * time: 11.586122989654541
    13    -6.211532e+01     1.439985e-02
 * time: 12.276822090148926
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557653
    AtomicNonlocal      14.8522624
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666457158
